## Cascading

In [27]:
import pandas as pd

# ===== 파일 경로 =====
OC_FILE  = "/home/eoil/AGENT/ICASSP2026/experiments/results/findings/ASV_thr/004/resnetOC.txt"
R34_FILE = "/home/eoil/AGENT/ICASSP2026/experiments/results/findings/ASV_thr/004/resnet34.txt"

# ===== Threshold 설정 =====
OC_THRESHOLD  = 0.99
R34_THRESHOLD = 0.42

# ===== 파일 로드 =====
# resnetOC.txt
df_oc = pd.read_csv(OC_FILE, delim_whitespace=True, header=None, names=["pair", "oc_score"])

# resnet34.txt
df_r34 = pd.read_csv(R34_FILE)

# key 만들기 (utt_id + "_" + enroll_id)
df_r34["pair"] = df_r34["utt_id"].astype(str) + "_" + df_r34["enroll_id"].astype(str)

# ===== merge =====
df = pd.merge(df_oc, df_r34, on="pair", how="inner")

# ===== 판정 =====
df["oc_pass"]  = df["oc_score"]      >= OC_THRESHOLD
df["r34_pass"] = df["cosine_score"]  >= R34_THRESHOLD
df["both_pass"] = df["oc_pass"] & df["r34_pass"]

# ===== 비율 계산 =====
total = len(df)
oc_rate   = df["oc_pass"].mean()
r34_rate  = df["r34_pass"].mean()
both_rate = df["both_pass"].mean()
none_rate = ((~df["oc_pass"]) & (~df["r34_pass"])).mean()

print(f"✅ Total pairs: {total}")
print(f"✅ OC pass rate:     {oc_rate*100:.2f}%")
print(f"✅ ResNet34 pass rate: {r34_rate*100:.2f}%")
print(f"✅ BOTH pass rate:   {both_rate*100:.2f}%")
print(f"✅ NONE pass rate:   {none_rate*100:.2f}%")  # 둘 다 못 넘긴 경우


✅ Total pairs: 4000
✅ OC pass rate:     76.58%
✅ ResNet34 pass rate: 18.77%
✅ BOTH pass rate:   14.35%
✅ NONE pass rate:   19.00%


## Score sum v1

In [59]:
import pandas as pd

# ===== 파일 경로 =====
OC_FILE  = "/home/eoil/AGENT/ICASSP2026/experiments/results/AGENT_ECA_AA/016/resnetOC.txt"
R34_FILE = "/home/eoil/AGENT/ICASSP2026/experiments/results/AGENT_ECA_AA/016/resnet34.txt"

# ===== 합산 Threshold 설정 =====
SUM_THRESHOLD = 1.51

# ===== 파일 로드 =====
# resnetOC.txt
df_oc = pd.read_csv(OC_FILE, delim_whitespace=True, header=None, names=["pair", "oc_score"])

# resnet34.txt
df_r34 = pd.read_csv(R34_FILE)

# key 만들기 (utt_id + "_" + enroll_id)
df_r34["pair"] = df_r34["utt_id"].astype(str) + "_" + df_r34["enroll_id"].astype(str)

# ===== merge =====
df = pd.merge(df_oc, df_r34, on="pair", how="inner")

# ===== 합산 점수 계산 =====
df["sum_score"] = df["oc_score"] + df["cosine_score"]

# ===== 합산 threshold 판정 =====
df["sum_pass"] = df["sum_score"] >= SUM_THRESHOLD

# ===== 비율 계산 =====
total = len(df)
pass_rate = df["sum_pass"].mean()
fail_rate = 1 - pass_rate

print(f"✅ Total pairs: {total}")
print(f"✅ SUM >= {SUM_THRESHOLD} pass rate: {pass_rate*100:.2f}%")


✅ Total pairs: 4000
✅ SUM >= 1.51 pass rate: 86.65%


## v2 ASR

In [ ]:
import pandas as pd
import numpy as np

# ===== 파일 경로 =====
OC_FILE  = "/home/eoil/AGENT/ICASSP2026/experiments/results/findings/ASV_thr/004/resnetOC.txt"
R34_FILE = "/home/eoil/AGENT/ICASSP2026/experiments/results/findings/ASV_thr/004/resnet34.txt"

# ===== 합산 Threshold 설정 =====
SUM_THRESHOLD = 1.27

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# ===== 파일 로드 =====
# resnetOC.txt
df_oc = pd.read_csv(OC_FILE, delim_whitespace=True, header=None, names=["pair", "oc_score"])

# resnet34.txt
df_r34 = pd.read_csv(R34_FILE)

# key 만들기 (utt_id + "_" + enroll_id)
df_r34["pair"] = df_r34["utt_id"].astype(str) + "_" + df_r34["enroll_id"].astype(str)

# ===== merge =====
df = pd.merge(df_oc, df_r34, on="pair", how="inner")

# ===== 합산 점수 계산 =====
df["sum_score"] = sigmoid(df["oc_score"]) + df["cosine_score"]

# ===== 합산 threshold 판정 =====
df["sum_pass"] = df["sum_score"] >= SUM_THRESHOLD

# ===== 비율 계산 =====
total = len(df)
pass_rate = df["sum_pass"].mean()
fail_rate = 1 - pass_rate

print(f"✅ Total pairs: {total}")
print(f"✅ SUM >= {SUM_THRESHOLD} pass rate: {pass_rate*100:.2f}%")


✅ Total pairs: 4000
✅ SUM >= 1.27 pass rate: 0.00%
